In [1]:
import numpy as np
from scipy.sparse import rand as sprand
import torch

# Make up some random explicit feedback ratings
# and convert to a numpy array
n_users = 1_000
n_items = 1_000
ratings = sprand(n_users, n_items, density=0.01, format="csr")
ratings.data = np.random.randint(1, 5, size=ratings.nnz).astype(np.float64)
ratings = ratings.toarray()
print(ratings.data)

In [2]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)


In [3]:
model = MatrixFactorization(n_users, n_items, n_factors=20)
print(model)

MatrixFactorization(
  (user_factors): Embedding(1000, 20, sparse=True)
  (item_factors): Embedding(1000, 20, sparse=True)
)


In [4]:
loss_func = torch.nn.MSELoss()
print(loss_func)

MSELoss()


In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)  # learning rate

In [6]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

for row, col in zip(*(rows, cols)):
    # Set gradients to zero
    optimizer.zero_grad()
    
    # Turn data into tensors
    rating = torch.FloatTensor([ratings[row, col]])
    row = torch.LongTensor([row])
    col = torch.LongTensor([col])

    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)
    
    # Backpropagate
    loss.backward()

    # Update the parameters
    optimizer.step()
    
    print(loss)


tensor(2.6416, grad_fn=<MseLossBackward0>)
tensor(5.6223, grad_fn=<MseLossBackward0>)
tensor(64.0698, grad_fn=<MseLossBackward0>)
tensor(6.0622, grad_fn=<MseLossBackward0>)
tensor(22.7122, grad_fn=<MseLossBackward0>)
tensor(8.7354, grad_fn=<MseLossBackward0>)
tensor(2.5765, grad_fn=<MseLossBackward0>)
tensor(0.0093, grad_fn=<MseLossBackward0>)
tensor(6.1219, grad_fn=<MseLossBackward0>)
tensor(74.0604, grad_fn=<MseLossBackward0>)
tensor(3.1351, grad_fn=<MseLossBackward0>)
tensor(0.5471, grad_fn=<MseLossBackward0>)
tensor(1.9299, grad_fn=<MseLossBackward0>)
tensor(141.2205, grad_fn=<MseLossBackward0>)
tensor(10.4845, grad_fn=<MseLossBackward0>)
tensor(59.6593, grad_fn=<MseLossBackward0>)
tensor(2.0227, grad_fn=<MseLossBackward0>)
tensor(0.5703, grad_fn=<MseLossBackward0>)
tensor(2.4834, grad_fn=<MseLossBackward0>)
tensor(95.5510, grad_fn=<MseLossBackward0>)
tensor(36.8675, grad_fn=<MseLossBackward0>)
tensor(1.4949, grad_fn=<MseLossBackward0>)
tensor(20.8101, grad_fn=<MseLossBackward0>)
t